In [15]:
# библиотека, умеющая управлять HTTP запросами (отправлять и получать ответы)
# library to manage HTTP requests
import requests

# библиотека для работы с JSON
# library to interact with JSON
import json

# библиотека для генерации случайных чисел
# library to generate random numbers
import random

In [16]:
# простейший пример GET запроса. Запишем в переменную r результат GET запроса к google.ru (как будто мы браузер)
# simples GET request example. Lets write to r variable result of request to google.ru (imagine that yoy are a web-browser)
r = requests.get('https://google.ru')

In [17]:
# Теперь в переменной r лежит всё, что связано с ответом на запрос.
# Например, код статуса:
# Now r variable contains everything about request
# For example, status code:
r.status_code

200

In [18]:
# Собственно тело ответа
# Content of response
r.content

b'<!doctype html><html itemscope="" itemtype="http://schema.org/WebPage" lang="ru"><head><meta content="\xcf\xee\xe8\xf1\xea \xe8\xed\xf4\xee\xf0\xec\xe0\xf6\xe8\xe8 \xe2 \xe8\xed\xf2\xe5\xf0\xed\xe5\xf2\xe5: \xe2\xe5\xe1 \xf1\xf2\xf0\xe0\xed\xe8\xf6\xfb, \xea\xe0\xf0\xf2\xe8\xed\xea\xe8, \xe2\xe8\xe4\xe5\xee \xe8 \xec\xed\xee\xe3\xee\xe5 \xe4\xf0\xf3\xe3\xee\xe5." name="description"><meta content="noodp" name="robots"><meta content="text/html; charset=UTF-8" http-equiv="Content-Type"><meta content="/logos/doodles/2021/get-vaccinated-wear-a-mask-save-lives-august-4-6753651837109295-law.gif" itemprop="image"><meta content="\xc7\xe0\xf9\xe8\xf9\xe0\xe9\xf2\xe5 \xf1\xe5\xe1\xff \xe8 \xe1\xeb\xe8\xe7\xea\xe8\xf5 #GoogleDoodle" property="twitter:title"><meta content="\xc7\xe0\xf9\xe8\xf9\xe0\xe9\xf2\xe5 \xf1\xe5\xe1\xff \xe8 \xe1\xeb\xe8\xe7\xea\xe8\xf5 #GoogleDoodle" property="twitter:description"><meta content="\xc7\xe0\xf9\xe8\xf9\xe0\xe9\xf2\xe5 \xf1\xe5\xe1\xff \xe8 \xe1\xeb\xe8\xe7\xe

In [5]:
# Перейдём к NGW. Решим задачу из 5-ти шагов:
# 1. Создать группу ресурсов
# 2. Создать в ней новый векторный слой
# 3. Создать внутри него несколько объектов
# 4. Создать стиль MapServer
# 5. Создать карту с этим слоем

# Lets take a look at NGW. Imagine workflow with 5 steps:
# 1. Create resource group (folder)
# 2. Create new vector layer
# 3. Create several vector objects
# 4. Create MapServer Style
# 5. Create Web Map with this layer

In [19]:
# Параметры подключения к Веб ГИС
# Options to connect to Web GIS
ngw_host = 'https://test-premium.nextgis.com'
ngw_user = 'webinar'
ngw_password = 'webinar00'
parent_resource = 259

In [23]:
# 1. Создать группу ресурсов
# 1. Create resource group
# https://docs.nextgis.ru/docs_ngweb_dev/doc/developer/create.html#resource-group

request_url = '%s/api/resource/' % ngw_host

# Структура нового объекта
# New object structure
data = {
    "resource":
    {
        "cls":"resource_group",
        "parent":
        {
            "id": parent_resource # родительский ресурс | parent resource
        },
        "display_name": u'Демонстрационная группа ресуров', # имя новой группы ресурсов | name for new resource group
        "description":""
    }
}

# отправляем запрос
# send requst
r = requests.post(request_url,
                  data=json.dumps(data),
                  auth=(ngw_user, ngw_password))

print (u'Статус (Status): %s' % r.status_code)
print (r.content)



Статус: 201
b'{"id": 339, "parent": {"id": 259}}'


In [24]:
new_resource_group_id = r.json()['id']

In [25]:
# 2. Создать в ней новый векторный слой
# 2. Create new vector layer
# https://docs.nextgis.ru/docs_ngweb_dev/doc/developer/create.html#empty-vector-layer
request_url = '%s/api/resource/' % ngw_host

# Структура нового объекта
# New object structure
data = {
    "resource":
    {
        "cls":"vector_layer",
        "parent":
        {
            "id": new_resource_group_id # родительский ресурс | parent resource
        },
        "display_name": u'Демонстрационный векторный слой', # имя нового векторного слоя | name of new vector layer
        "description":""
    },
    "resmeta":{
      "items":{}
    },
    "vector_layer":{
        "srs":{ "id":3857 },
        "geometry_type": "POINT",
        "fields": [
            {
                "keyname": "Number",
                "datatype": "INTEGER",
                "display_name": u"Номер"
            },
            {
                "keyname": "Name",
                "datatype": "STRING",
                "display_name": u"Название"
            }
        ]
    }
}

# отправляем запрос
# send request
r = requests.post(request_url,
                  data=json.dumps(data),
                  auth=(ngw_user, ngw_password))

print (u'Статус (Status): %s' % r.status_code)
print (r.content)

Статус: 201
b'{"id": 340, "parent": {"id": 339}}'


In [26]:
new_vector_layer_id = r.json()['id']

In [27]:
# 3. Создать внутри него несколько объектов
# 3. Create several vector objects with random coordinates
# https://docs.nextgis.ru/docs_ngweb_dev/doc/developer/create.html#feature-in-vector-or-postgis-layer

request_url = '%s/api/resource/%s/feature/' % (ngw_host, new_vector_layer_id)

for i in range(0,3):
    # Структура нового объекта
    # new object structure
    data = {
        "extensions": {
            "attachment": None,
            "description": None
        },
        "fields": {
            "Number": i,
            "Name": u"Пример (example) %s" % i
        },
        "geom": "POINT (%s %s)" % (random.randint(15112000,15114000), random.randint(6057000,6059000))
    }

    # отправляем запрос
    # send request
    r = requests.post(request_url,
                      data=json.dumps(data),
                      auth=(ngw_user, ngw_password))

    print (i)
    print (u'Статус (Status): %s' % r.status_code)
    print (r.content)

0
Статус: 200
b'{"id": 1}'
1
Статус: 200
b'{"id": 2}'
2
Статус: 200
b'{"id": 3}'


In [28]:
# 4. Создать стиль MapServer
# 4. Create MapServer style
# https://docs.nextgis.ru/docs_ngweb_dev/doc/developer/create.html#mapserver-style

request_url = '%s/api/resource/' % ngw_host

data = {
    "mapserver_style" : {
    "xml" : "<map><symbol><type>ellipse</type><name>circle</name><points>1 1</points><filled>true</filled></symbol><layer><class><style><color blue=\"98\" green=\"180\" red=\"253\"/><outlinecolor blue=\"64\" green=\"64\" red=\"64\"/><symbol>circle</symbol><size>6</size></style></class></layer><legend><keysize y=\"15\" x=\"15\"/><label><size>12</size><type>truetype</type><font>regular</font></label></legend></map>"
    },
    "resource": {
        "cls": "mapserver_style",
        "description": None,
        "display_name": u"Пример стиля (style example)",
        "keyname": None,
        "parent": {
            "id": new_vector_layer_id
        }
    }
}

# отправляем запрос
# send request
r = requests.post(request_url,
                  data=json.dumps(data),
                  auth=(ngw_user, ngw_password))

print (i)
print (u'Статус: %s' % r.status_code)
print (r.content)

2
Статус: 201
b'{"id": 341, "parent": {"id": 340}}'


In [29]:
new_layer_style_id = r.json()['id']

In [30]:
# 5. Создать карту с этим слоем
# 5. Create web map with this layer
# https://docs.nextgis.ru/docs_ngweb_dev/doc/developer/create.html#web-map

request_url = '%s/api/resource/' % ngw_host

data = {
        "resource": {
            "display_name": u"Пример карты (map example)",
            "parent": {
                "id": new_resource_group_id
            },
            "cls": "webmap"
        },
        "webmap": {
            "root_item": {
                "item_type": "root",
                "children": [
                    {
                        "layer_enabled": True,
                        "layer_adapter": "image",
                        "display_name": u'Слой на карте (layer on map)',
                        "layer_style_id": new_layer_style_id,
                        "item_type": "layer"
                    }
                ]
            }
        }
    }

# отправляем запрос
# send request
r = requests.post(request_url,
                  data=json.dumps(data),
                  auth=(ngw_user, ngw_password))

print (i)
print (u'Статус (Status): %s' % r.status_code)
print (r.content)

2
Статус: 201
b'{"id": 342, "parent": {"id": 339}}'
